In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import sys
import ROOT
from ROOT import (RooArgSet,
                  RooCategory,
                  RooRealVar,
                  RooDataSet,
                  RooPlot,
                  RooFit,
                  RooStats,
                  RooArgList,
                  RooFormulaVar)

from ROOT import (TFile, 
                  TH1D, 
                  TH2D, 
                  TCanvas, 
                  gStyle, 
                  TLine, 
                  TTree)

import math
import itertools
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from root_pandas import read_root
import pandas as pd
            
sys.path.append('/home/vmueller/repos/')
from dopy.doroot.root_utils import read_roodataset_from_tree, plot_pulls, build_tchain_from_files
from dopy.doplot.plotting import Plotter, Plot
import dopy.doroot.rootnotes 
from dopy.doplot.selection import add_log_to_dataframe, add_max_to_dataframe, add_min_to_dataframe

In [ ]:
triggerlines_ = '(Hlt1DiMuonHighMass==1|Hlt1TrackMuon==1)&&Hlt2DiMuonDetachedJPsi==1'

### Sanity cuts:  



In [11]:
sanity_cuts = [ 'B0_FitPVConst_MERR_flat<30',
        'B0_FitPVConst_tauErr_flat<0.0002',
        'PVZ<200',
        '(KS0_OWNPV_CHI2/KS0_OWNPV_NDOF)<20',
        '(J_psi_1S_OWNPV_CHI2/J_psi_1S_OWNPV_NDOF)<16',
        '(B0_OWNPV_CHI2/B0_OWNPV_NDOF)<10',
        'B0_FitPVConst_J_psi_1S_P1_PT>500',
        'piminus_TRACK_CHI2NDOF<3',
        'muminus_PIDmu>0',
        'B0_FitPVConst_KS0_P0_PT_flat>2000',
        '(piminus_MINIPCHI2/piminus_OWNPV_NDOF)>4',
        'B0_FitJpsiPVConst_status==0',
        'B0_FitDaughtersPVConst_status==0',
        'B0_FitPVConst_status==0']

In [12]:
s = '&'
s.join(sanity_cuts)

'B0_FitPVConst_MERR_flat<30&B0_FitPVConst_tauErr_flat<0.0002&PVZ<200&(KS0_OWNPV_CHI2/KS0_OWNPV_NDOF)<20&(J_psi_1S_OWNPV_CHI2/J_psi_1S_OWNPV_NDOF)<16&(B0_OWNPV_CHI2/B0_OWNPV_NDOF)<10&B0_FitPVConst_J_psi_1S_P1_PT>500&piminus_TRACK_CHI2NDOF<3&muminus_PIDmu>0&B0_FitPVConst_KS0_P0_PT_flat>2000&B0_FitJpsiPVConst_status==0&B0_FitDaughtersPVConst_status==0&B0_FitPVConst_status==0'

## Daughter cuts 

In [15]:
daughter_cuts = [
                'abs(B0_FitDaughtersConst_J_psi_1S_M_flat-3096.916)<60',
                '((abs(B0_FitDaughtersConst_KS0_M_flat_1S_M-497.614)<55&piminus_TRACK_type==5)|(abs(B0_FitDaughtersConst_KS0_M_flat_1S_M-497.614)<15&piminus_TRACK_type==3))'
    
                ]

In [16]:
s = '&'
s.join(daughter_cuts)

'abs(B0_FitDaughtersConst_J_psi_1S_M_flat-3096.916)<60&((abs(B0_FitDaughtersConst_KS0_M_flat_1S_M-497.614)<55&piminus_TRACK_type==5)|(abs(B0_FitDaughtersConst_KS0_M_flat_1S_M-497.614)<15&piminus_TRACK_type==3))'